In [85]:
import os
import numpy as np
from tqdm.notebook import tqdm
import librosa
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

from xgboost import XGBClassifier

Для обучения возьмем 2 датасета: dev-clean.tar.gz и dev-other.tar.gz (more challenging), объединив их в один
Для тестирования соответственно: test-clean.tar.gz и test-other.tar.gz 

In [67]:
TRAIN_DATA_DIR = '/Users/a.gorlenko/made/speech/train_data'
TEST_DATA_DIR = '/Users/a.gorlenko/made/speech/test_data'

In [68]:
N_MFCC = 13
N_FFT_COEF = 0.025
HOP_LENGTH_COEF = 0.01

In [69]:
def get_gender(reader_df, reader_id):
    return reader_df.loc[reader_id]['GENDER']

In [70]:
def find_audio_files(data_dir):
    audio_files = []
    for file_dir, _, files in os.walk(data_dir):
        for file in files:
            if file.endswith('.wav'):
                audio_files.append((file_dir, file))
    return audio_files

Преобразуем аудиофайлы (wav) из указанной директории (и ее поддиректорий) в данные для обучения.
В качестве фичей возьмем:
- Мел-кепстральные коэффициенты mfcc
- данные по высоте звука (мужской голос, как правило, ниже)

Изначально получаем для каждого файла набор векторов, но хочется работать с одним векторов для файла. Поэтому сагрегируем данные для каждого файла. Для mfcc возьмем:
- среднее
- дисперсию
- минимум
- максимум

Как показали опыты, каждый из этих показателей дает прибавку в качестве.

Для высоты возьмем просто среднее (другое не дало особой прибавки).

In [73]:
def build_df(data_dir):
    X = []
    y = []
    
    reader_df = pd.read_csv(os.path.join(data_dir, 'LibriTTS', 'speakers.tsv'), 
                            sep='\t', 
                            names=['READER', 'GENDER', 'SUBSET', 'NAME'], 
                            header=0, 
                            index_col=0)
    
    audio_files = find_audio_files(data_dir)
    for dir_path, file_name in tqdm(audio_files):
        reader_id = int(file_name.split('_')[0])
        audio_data, sr = librosa.load(os.path.join(dir_path, file_name), sr=None)
        mfcc = librosa.feature.mfcc(y=audio_data, 
                                    sr=sr, 
                                    n_mfcc=N_MFCC, 
                                    n_fft=int(sr * N_FFT_COEF), 
                                    hop_length=int(sr * HOP_LENGTH_COEF))
        pitches, magnitudes = librosa.piptrack(y=audio_data, 
                                               sr=sr, 
                                               n_fft=int(sr * N_FFT_COEF), 
                                               hop_length=int(sr * HOP_LENGTH_COEF))
        X.append(np.concatenate((
            np.mean(mfcc, axis=1), 
            np.std(mfcc, axis=1), 
            np.max(mfcc, axis=1), 
            np.min(mfcc, axis=1),
            np.mean(pitches, axis=1),
        )))
        y.append(get_gender(reader_df, reader_id))
    return np.array(X), np.array(y) == 'F'

In [74]:
X, y = build_df(TRAIN_DATA_DIR)

  0%|          | 0/10349 [00:00<?, ?it/s]

Убедимся, что имеем дело со сбалансированными классами:

In [75]:
np.unique(y, return_counts=True)

(array([False,  True]), array([4932, 5417]))

Попробуем проверить на кроссвалидации несколько моделей классификаторов:

In [78]:
rfc = RandomForestClassifier(random_state=42)
svc = SVC(probability=True, random_state=42)
xgb = XGBClassifier(n_estimators=500, random_state=42)

In [79]:
classifiers = [
    ('random forest', rfc),
    ('svc', svc),
    ('xgboost', xgb)
]

In [81]:
for classifier_name, classifier in classifiers:
    scores = cross_val_score(classifier, X, y, scoring='roc_auc', cv=3)
    print(f'{classifier_name}: {np.mean(scores)}')

random forest: 0.9254543458795297
svc: 0.9297284380775866
xgboost: 0.9467499390184342


В целом даже без особого подбора гиперпараметров все классификаторы показали неплохие результаты.

Посмотрим результаты xgboost на тесте:

In [83]:
xgb.fit(X, y)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=500,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=42,
              reg_alpha=0, reg_lambda=1, ...)

In [82]:
X_test, y_test = build_df(TEST_DATA_DIR)

  0%|          | 0/9957 [00:00<?, ?it/s]

In [84]:
roc_auc_score(y_test, xgb.predict_proba(X_test)[:, 1])

0.9520462549580319

In [88]:
accuracy_score(y_test, xgb.predict(X_test))

0.893140504167922

In [89]:
f1_score(y_test, xgb.predict(X_test))

0.9028487947406866

Видим, что использование mfcc и высоты голоса в качестве фичей позволяют добиться достаточно неплохого качества в классификации мужского/женского голоса даже без тщательного подбора геперпараметров и использования сложных моделей (например, нейросетей).